## Read sample COCO img

In [1]:
import cv2
import IPython

In [2]:
# if input image is in range 0..1, please first multiply img by 255
# assume image is ndarray of shape [height, width, channels] where channels can be 1, 3 or 4
def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [3]:
img = cv2.imread('000000000001.jpg')

## Load same faster rcnn model as in VAS, run prediction on img

In [4]:
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

import torch

In [5]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

model = build_model(cfg).eval()

DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

outputs = model(torch.from_numpy(img).permute(2,0,1).unsqueeze(0))

In [6]:
from modified_rcnn import ModifiedGeneralizedRCNN
device = 'cuda'
modified = ModifiedGeneralizedRCNN(model).to(device).eval()

/home/reedless/detectron2/detectron2/modeling/meta_arch/rcnn.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("pixel_mean", torch.tensor(pixel_mean).view(-1, 1, 1), False)
/home/reedless/detectron2/detectron2/modeling/meta_arch/rcnn.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("pixel_std", torch.tensor(pixel_std).view(-1, 1, 1), False)


In [8]:
DetectionCheckpointer(modified).load(cfg.MODEL.WEIGHTS)
results = modified.inference(torch.from_numpy(img).permute(2,0,1).unsqueeze(0), do_postprocess=False, class_scores_only=True)

TypeError: inference() takes 3 positional arguments but 4 were given

In [ ]:
results.shape

In [ ]:
img.shape, torch.from_numpy(img).permute(2,0,1).shape

In [ ]:
outputs[0]['instances'][0].pred_classes[0]

In [ ]:
outputs[0]['instances']

In [ ]:
outputs[0]['instances'].pred_boxes.tensor.cpu().detach().int().numpy()

In [ ]:
import numpy as np

clone = img.copy()

for box in outputs[0]['instances'].pred_boxes.tensor.cpu().detach().int().numpy():
    x0, y0, x1, y1 = box
    clone[y0:y1, x0] = np.full((y1-y0, 3), 255)
    clone[y0:y1, x1] = np.full((y1-y0, 3), 255)
    clone[y0, x0:x1] = np.full((x1-x0, 3), 255)
    clone[y1, x0:x1] = np.full((x1-x0, 3), 255)

imshow(clone)

### Visualise output

In [ ]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs[0]["instances"].to("cpu"))
imshow(out.get_image()[:, :, ::-1])

## Partially executing faster rcnn

### Feature extraction

In [ ]:
input_img = torch.from_numpy(img).permute(2,0,1).unsqueeze(0).float().to("cuda")

images = model.preprocess_image(input_img)

# features
features = model.backbone(images.tensor)

In [ ]:
features.keys()

In [ ]:
features['p2'].shape, features['p6'].shape

#### Global Average Pool feature maps

In [ ]:
m = torch.nn.AvgPool3d(features['p2'].shape[1:])
print(m(features['p2'])[0, 0, 0, 0])
m = torch.nn.AvgPool3d(features['p6'].shape[1:])
print(m(features['p6'])[0, 0, 0, 0])

### Features to RPN proposals

In [ ]:
pred_objectness_logits, pred_anchor_deltas = model.proposal_generator.rpn_head([features[f] for f in cfg.MODEL.RPN.IN_FEATURES])

In [ ]:
pred_objectness_logits[0].shape, pred_objectness_logits[4].shape

In [ ]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(2, 3))

for i in range(len(pred_objectness_logits)):
    heatmap = (pred_objectness_logits[i]
               .permute(0, 2, 3, 1)
               .cpu().detach()
               .apply_(sigmoid)[0,:,:,1])
                # 3 types of boxes, 1 is square, or 1:1 boxes
    fig.add_subplot(2, 3, i+1)
    plt.imshow(img)
    scale = int(img.shape[0]/heatmap.shape[0])
    plt.imshow(np.kron(heatmap, np.ones((scale, scale))), cmap=plt.get_cmap('jet'), alpha=0.4)
    
fig.set_size_inches(18, 10)

### Proposal Generator

In [ ]:
# proposals
proposals, losses = model.proposal_generator(images, features)

In [ ]:
model.proposal_generator

In [ ]:
proposals[0] # proposal_boxes, objectness_logits

In [ ]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

objectness_scores = proposals[0].objectness_logits.cpu().apply_(sigmoid)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

clone = np.zeros(img.shape[:2])
boxes = proposals[0].proposal_boxes.tensor.cpu().detach().int().numpy()

for i in range(len(boxes)):
    box = boxes[i]
    x0, y0, x1, y1 = box
    if x1 == 640:
        x1 -= 1
    if y1 == 480:
        y1 -= 1
    clone[y0:y1, x0:x1] += np.full((y1-y0, x1-x0), objectness_scores[i])
    
plt.imshow(img)
plt.imshow(clone, cmap=plt.get_cmap('jet'), alpha=0.5)

### ROI

In [ ]:
type(model.roi_heads)

In [ ]:
features_list = [features[f] for f in model.roi_heads.box_in_features]
model.roi_heads.box_in_features, features.keys() # note the missing 'p6' from features.keys()

In [ ]:
box_features = model.roi_heads.box_pooler(features_list, [x.proposal_boxes for x in proposals])
box_features.shape

#### Some examples of cropped ROI

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(1, 5))

for i in range(5):
    cropped_roi = torch.sum(box_features[0].cpu().detach(), dim=0)
    fig.add_subplot(1, 5, i+1)
    plt.imshow(cropped_roi)
    
fig.set_size_inches(10, 5)

#### Flatten cropped ROI

In [ ]:
model.roi_heads.box_head

In [ ]:
box_features2 = model.roi_heads.box_head(box_features)
box_features2, box_features2.shape

#### Convert flattened and linear layers to class scores and proposal deltas

In [ ]:
model.roi_heads.box_predictor.cls_score.in_features

In [ ]:
scores, proposal_deltas = predictions = model.roi_heads.box_predictor(box_features2)
scores.shape, proposal_deltas.shape

In [ ]:
pred_instances, _ = model.roi_heads.box_predictor.inference(predictions, proposals)
pred_instances

#### Break down model.roi_heads.box_predictor.inference

In [ ]:
'''
list[Tensor]:
A list of Tensors of predicted class-specific or class-agnostic boxes
for each image. Element i has shape (Ri, K * B) or (Ri, B), where Ri is
the number of proposals for image i and B is the box dimension (4 or 5)
'''
# basically converts proposal_deltas to actual bounding boxes
boxes = model.roi_heads.box_predictor.predict_boxes(predictions, proposals)
boxes[0].shape # per ROI, we have 80 class-specific bounding boxes

In [ ]:
'''
list[Tensor]:
A list of Tensors of predicted class probabilities for each image.
Element i has shape (Ri, K + 1), where Ri is the number of proposals for image i.
'''
# basically F.softmax(scores, dim=-1) for all
scores = model.roi_heads.box_predictor.predict_probs(predictions, proposals)
scores[0].shape

In [ ]:
# len(proposals) = len(image_shapes) = 1
image_shapes = [x.image_size for x in proposals]
image_shapes

In [ ]:
from typing import Dict, List, Tuple, Union

def fast_rcnn_inference(
    boxes: List[torch.Tensor],
    scores: List[torch.Tensor],
    image_shapes: List[Tuple[int, int]],
    score_thresh: float, # set by config, is 0.5 rn
    nms_thresh: float, # 0.5 by default
    topk_per_image: int, # 100 by default, but usually won't have that many detected objects in an image
):
    
    # result_per_image : list[(Instances, Tensor)]
    result_per_image = [
        fast_rcnn_inference_single_image(
            boxes_per_image, scores_per_image, image_shape, score_thresh, nms_thresh, topk_per_image
        )
        for scores_per_image, boxes_per_image, image_shape in zip(scores, boxes, image_shapes)
    ]
    
    # list[Instances], list[Tensor]
    return [x[0] for x in result_per_image], [x[1] for x in result_per_image]

# TODO

Edit fast_rcnn_inference_single_image on jpnote to return array of instances with associated class probabilities DONE

Copy changes to subclass of detectron2.modeling.roi_heads.fast_rcnn.fast_rcnn_inference

### Think...

Should I subclass ModifiedGeneralizedRCNN some more to redifine fast_rcnn_inference_single_image?

In [ ]:
from detectron2.structures import Boxes, Instances
from detectron2.layers import batched_nms

def fast_rcnn_inference_single_image(
    boxes,
    scores,
    image_shape: Tuple[int, int],
    score_thresh: float,
    nms_thresh: float,
    topk_per_image: int,
):
    """
    Single-image inference. Return bounding-box detection results by thresholding
    on scores and applying non-maximum suppression (NMS).
    """
    
    # sanity check that all values are finite, i.e. they are not NaN, negative infinity, or infinity
    valid_mask = torch.isfinite(boxes).all(dim=1) & torch.isfinite(scores).all(dim=1)
    if not valid_mask.all():
        boxes = boxes[valid_mask]
        scores = scores[valid_mask]

    scores = scores[:, :-1] # drop last class as that is background (class 80)
    raw_scores = scores.detach().clone()    
    
    # total number of classes (80)
    num_bbox_reg_classes = boxes.shape[1] // 4
    
    # Convert to Boxes to use the `clip` function
    '''
    .reshape(-1, 4) to flatten out the 80 class-specific bounding boxes per ROI 
        into 80*NUM_ROI w NUM_ROI=1000 boxes
    '''
    boxes = Boxes(boxes.reshape(-1, 4)) 
    '''
    Clip (in place) the boxes by limiting x coordinates to the range [0, width]
        and y coordinates to the range [0, height].
    '''
    boxes.clip(image_shape)
    '''
    .view(-1, num_bbox_reg_classes, 4) to convert back to NUM_ROI * 80 * 4
    Somehow the order doesn't get mixed up lol works as expected :O
    '''
    boxes = boxes.tensor.view(-1, num_bbox_reg_classes, 4)

    # 1. Filter results based on detection scores. It can make NMS more efficient
    #    by filtering out low-confidence detections.
    filter_mask = scores > score_thresh  # R x K w R = 1000, K = 80
    
    # R' x 2. First column contains indices of the R predictions;
    # Second column contains indices of classes.
    # R' <= R
    filter_inds = filter_mask.nonzero()
    
    if num_bbox_reg_classes == 1:
        boxes = boxes[filter_inds[:, 0], 0]
    else:
        boxes = boxes[filter_mask] # [R', 4]
        
    scores = scores[filter_mask] # [R']
    raw_scores = raw_scores[filter_inds[:,0]] # [R', K] w K = 80
    print(raw_scores.shape)

    # 2. Apply NMS for each class independently.
    keep = batched_nms(boxes, scores, filter_inds[:, 1], nms_thresh)
    
    if topk_per_image >= 0:
        keep = keep[:topk_per_image]
        # int64 tensor with the indices of the elements that 
        # have been kept by NMS, sorted in decreasing order of scores
        
    boxes, scores, filter_inds, raw_scores = boxes[keep], scores[keep], filter_inds[keep], raw_scores[keep]
    print(raw_scores.shape)

    # convert results into Instances
    result = Instances(image_shape)
    result.pred_boxes = Boxes(boxes)
    result.scores = scores
    result.pred_classes = filter_inds[:, 1]
    result.raw_scores = raw_scores
    
    # Instances, Tensor
    return result, filter_inds[:, 0]

In [ ]:
fast_rcnn_inference(
            boxes,
            scores,
            image_shapes,
            score_thresh=0.5,
            nms_thresh=0.5,
            topk_per_image=10,
        )

Only step left is post processing, which does not change any of the values but just presents the data in a different manner.

### Compare

In [ ]:
model._postprocess(pred_instances, input_img, images.image_sizes)

In [ ]:
outputs

## Captum

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from captum.attr import (
    GradientShap,
    DeepLift,
    DeepLiftShap,
    IntegratedGradients,
    LayerConductance,
    NeuronConductance,
    NoiseTunnel,
)

## Run on model

In [ ]:
input_   = torch.from_numpy(img).permute(2,0,1).unsqueeze(0)
baseline = torch.zeros(input_.shape)
baseline.shape

### Need input_/baseline to be tensor, model needs to take in individual tensor

In [ ]:
def wrap(input_tensor):
    print(input_tensor.shape)
    outputs = model(input_tensor)
    print(outputs)
    return outputs[0]['instances'][selected_pred].pred_classes[0]

In [ ]:
ig = IntegratedGradients(wrap)
attributions, delta = ig.attribute(input_, baseline, target=0, return_convergence_delta=True)
print('IG Attributions:', attributions)
print('Convergence Delta:', delta)

In [ ]:
gs = GradientShap(model)

# We define a distribution of baselines and draw `n_samples` from that
# distribution in order to estimate the expectations of gradients across all baselines
baseline_dist = torch.randn(5, 3, 480, 640) * 0.001
attributions, delta = gs.attribute(input_, stdevs=0.09, n_samples=4, baselines=baseline_dist,
                                   target=0, return_convergence_delta=True)
print('GradientShap Attributions:', attributions)
print('Convergence Delta:', delta)
print('Average delta per example:', torch.mean(delta.reshape(input.shape[0], -1), dim=1))

In [ ]:
dl = DeepLift(model)
attributions, delta = dl.attribute(input_, baseline, target=0, return_convergence_delta=True)
print('DeepLift Attributions:', attributions)
print('Convergence Delta:', delta)

In [ ]:
dl = DeepLiftShap(model)
attributions, delta = dl.attribute(input_.float(), baseline_dist, target=0, return_convergence_delta=True)
print('DeepLiftSHAP Attributions:', attributions)
print('Convergence Delta:', delta)
print('Average delta per example:', torch.mean(delta.reshape(input.shape[0], -1), dim=1))

In [ ]:
ig = IntegratedGradients(model)
nt = NoiseTunnel(ig)
attributions, delta = nt.attribute(input_, nt_type='smoothgrad', stdevs=0.02, nt_samples=4,
      baselines=baseline, target=0, return_convergence_delta=True)
print('IG + SmoothGrad Attributions:', attributions)
print('Convergence Delta:', delta)
print('Average delta per example', torch.mean(delta.reshape(input.shape[0], -1), dim=1))

In [ ]:
model

In [ ]:
nc = NeuronConductance(model, model.backbone)
attributions = nc.attribute(input_, neuron_selector=1, target=0)
print('Neuron Attributions:', attributions)

In [ ]:
lc = LayerConductance(model, model.backbone)
attributions, delta = lc.attribute(input_, baselines=baseline, target=0, return_convergence_delta=True)
print('Layer Attributions:', attributions)
print('Convergence Delta:', delta)

# Toy Model

In [ ]:
class ToyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(3, 3)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(3, 2)

        # initialize weights and biases
        self.lin1.weight = nn.Parameter(torch.arange(-4.0, 5.0).view(3, 3))
        self.lin1.bias = nn.Parameter(torch.zeros(1,3))
        self.lin2.weight = nn.Parameter(torch.arange(-3.0, 3.0).view(2, 3))
        self.lin2.bias = nn.Parameter(torch.ones(1,2))

    def forward(self, input):
        return self.lin2(self.relu(self.lin1(input)))

In [ ]:
model = ToyModel()
model.eval()

In [ ]:
toy_input = torch.rand(2, 3)
model(toy_input)